In [1]:
#pip install rank_bm25

In [2]:
#pip install wordcloud

In [3]:
#!python -m pip install -U spacy
#!python -m pip install scispacy 
#!python -m pip install "https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bionlp13cg_md-0.5.1.tar.gz"

In [4]:
#mandatory imports
import time
import re
from Bio import Entrez
import pandas as pd
import numpy as np
import urllib.request, urllib.error, urllib.parse
import json
#used for visualizations
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
#used for ranking and tokenization
from rank_bm25 import BM25Okapi
import spacy 
import scispacy 
import en_ner_bionlp13cg_md #The model we are going to use from spacy import displacy from scispacy.abbreviation
from scispacy.abbreviation import AbbreviationDetector 
from scispacy.umls_linking import UmlsEntityLinker

In [5]:
## Thesaurus declaration
thesaurus = {
    "Ivermectin" : ["IVM", "Ivermectin", "Avermectin", "avermectin", "ivermectin", "stromectol", "Stromectol", "Eqvalan", "Ivomec", "Mectizan", "Dihydroavermectin", "MK 933", "MK-933", "MK933","MK-0933", "L 640471", "L-640471", "C48H74O14", "IV", "IVM-654", "IVR-25", "IV-104", "IVE-11", "IVER-15"],
    "GABA" : ["GABA", "GABAergic", "gamma-aminobutyric acid"],
    "Zebrafish" : ["Zebrafish", "Danio rerio"],
    "COVID-19" : ["COVID-19", "COVID", "SARS-CoV-2"],
    "Glutamate" : ["Glutamate", "glutamate","Glu", "L-(+)-glutamate","L-Glu", "L-Glutamate", "L-glutamate", "L glutamate", "L glutamate"]
}
print("Done.")

Done.


In [6]:
def global_db_search(query_list, genes_list=[]):

    if len(genes_list) != 0:
        for i in query_list:
            for j in genes_list:
                query = i + j + "[tiab]"

                handle = Entrez.egquery(term=query)
                record = Entrez.read(handle)
                df = pd.DataFrame(record["eGQueryResult"]).head(2)
                df["Query"] = query
                append_data(df, 'global_query_res.csv', False)
                time.sleep(0.34)
    else:
        for i in query_list:
            handle = Entrez.egquery(term=i)
            record = Entrez.read(handle)
            df = pd.DataFrame(record["eGQueryResult"]).head(2)
            df["Query"] = i
            append_data(df, 'global_query_res.csv', False)
            time.sleep(0.34)
    return


## Function for reading in the df "summary" results
def read_in_results(file_name):

    # The converters are there so that each list is NOT inside a string
    res_df = pd.read_csv(file_name,  converters={"MainID_List": lambda x: x.strip("[]").replace("'", "").split(", "),
                                                    "P_Dates": lambda x: x.strip("[]").replace("'", "").split(", "),
                                                    "P_Years": lambda x: x.strip("[]").replace("'", "").split(", "),
                                                    "LinkedID_List": lambda x: x.strip("[]").replace("'", "").split(", "),
                                                    "Query_Count": int})
    return res_df


def esummary_info(in_webenv_key, in_query_key, db_name):

    # Obtaining DocSums for a set of IDs that are stored on the Entrez History server.
    handle = Entrez.esummary(db=db_name, webenv=in_webenv_key, query_key=in_query_key)
    record = Entrez.read(handle)

    publ_dates, publ_years = get_published_dates(record)

    if db_name == "pubmed":
        ids_list = get_pmcids(record)
    else:
        ids_list = get_pmids(record)

    return publ_dates, publ_years, ids_list


def get_published_dates(esummary_rec):

    retr_dates = []
    retr_years = []
    check = True
    i = 0
    for article in esummary_rec:
        # "PubDate" is often of the form: '2021 Nov 26'
        retr_dates.append(article["PubDate"])
        date = article["PubDate"].split()
        while(i < 2):
            if len(date[i]) == 4:
                p_year = int(date[i])
                i = 3
            else:
                i = i + 1
        #p_year = int(article["PubDate"].split()[0])
        #p_year = int(article["P_Years"].split()[0])
        #changed from PubDate to P_Years because of error when changing search term
        retr_years.append(p_year)

    return retr_dates, retr_years


def get_pmcids(esummary_rec):

    pmcids_list = []
    for i in esummary_rec:
        # If "pmc" is there, then this article also has a PMCID (i.e., it's also found in the PubMed Central db)
        if "pmc" in i["ArticleIds"]:
            pmcids_list.append(i["ArticleIds"]["pmc"])
        else:
            pmcids_list.append(np.NaN)

    return pmcids_list


def get_pmids(esummary_rec):

    pmids_list = []
    for i in esummary_rec:
        # '0' means that the article has no PMID (i.e., it's not found in the PubMed db)
        if i["ArticleIds"]["pmid"] == '0':
            pmids_list.append(np.NaN)
        else:
            pmids_list.append(i["ArticleIds"]["pmid"])

    return pmids_list


## Function that retrieves summary results from a given set of queries (which don't require a gene list)
def get_query_info_no_genes(query_in, db_name):

    # relevance: Records are sorted based on relevance to your search. (Relevance ranking)
    search_results = Entrez.read(
        Entrez.esearch(db=db_name, term=query_in, sort="relevance", retmax=5000, usehistory="y")
        )

    # NEED TO FIRST CHECK IF WE GOT ANY RESULTS FROM THAT QUERY
    if len(search_results["IdList"]) == 0:
        print("No Results.")
        return
    else:
        # With search_results, we will use its WebEnv value and QueryKey value
        p_dates, p_years, ids_list = esummary_info(search_results["WebEnv"], search_results["QueryKey"], db_name)

        time.sleep(0.34)

        return pd.DataFrame([[query_in, db_name, search_results['Count'], search_results['IdList'], p_dates, p_years, ids_list]],
                                columns=['Query', 'Db_Name', 'Query_Count', 'MainID_List', 'P_Dates', 'P_Years', 'LinkedID_List'])


## Function that retrieves summary results from a given set of queries (which requires a gene list)
def get_query_info(query_in, genes, db_name):

    gene_query = []
    query = ""

    for i in genes:
        # Example of db_name values in this use case: "pubmed" or "pmc"
        if db_name == "pubmed":
            # PubMed's Search field tag: Title/Abstract [tiab]
            query = query_in + i + "[tiab]"
        else:
            query = query_in + i

        # relevance: Records are sorted based on relevance to your search. (Relevance ranking)
        search_results = Entrez.read(
            Entrez.esearch(db=db_name, term=query, sort="relevance", retmax=5000, usehistory="y")
            )
     
        # NEED TO FIRST CHECK IF WE GOT ANY RESULTS FROM THAT QUERY
        if len(search_results["IdList"]) == 0:
            continue

        # With search_results, we will use its WebEnv value and QueryKey value
        p_dates, p_years, ids_list = esummary_info(search_results["WebEnv"], search_results["QueryKey"], db_name)
       
        gene_query.append([query, db_name, search_results['Count'], search_results['IdList'], p_dates, p_years, ids_list])
        time.sleep(0.34)
          
    return pd.DataFrame(gene_query, columns=['Query', 'Db_Name', 'Query_Count', 'MainID_List', 'P_Dates', 'P_Years', 'LinkedID_List'])


## Function for obtaining citation counts for the set of IDs found in the "summary" df
def cited_cnt_table(df_summary, db_name):

    elink_data = []
    link_name = ""

    if db_name == "pubmed":
        link_name = "pubmed_pubmed_citedin"
    else:
        link_name = "pmc_pmc_citedby"  # "pmc" is the other db_name in this use case

    for i in range(0, len(df_summary)):

        query_term = df_summary.iloc[i]["Query"]

        for id_num in df_summary.iloc[i]["MainID_List"]:

            record = Entrez.read(Entrez.elink(id=id_num, dbfrom=db_name, db=db_name, linkname=link_name))
         
            if len(record[0]["LinkSetDb"]) != 0:
                cited_counts = len(record[0]["LinkSetDb"][0]["Link"])
            else:
                # 'LinkSetDb' key contains empty list when an article has no citation counts
                cited_counts = 0
            elink_data.append([query_term, db_name, id_num, cited_counts])

            if (df_summary.iloc[i]["MainID_List"].index(id_num) + 1) % 3 == 0:
                time.sleep(0.34)

    return pd.DataFrame(elink_data, columns=["Query", "Db_Name", "Id_List", "Citation_Cnts"]) 


## Function that returns the Top-k results (pass in k as an argument to the function, input by the user)
def get_top_k(df, k_val):

    q_top_k = []

    for q in df["Query"].unique():
        matches_ids = []  # For each query version, these are the IDs meeting the criteria of having citation counts >= 25
        counts = []
        df_temp = df[df["Query"] == q]

        for i in range(0, len(df_temp)):
            if df_temp.iloc[i]["Citation_Cnts"] >= 25:
                matches_ids.append(int(df_temp.iloc[i]["Id_List"]))
                counts.append(df_temp.iloc[i]["Citation_Cnts"])
                if len(matches_ids) == k_val:
                    break
        if len(matches_ids) == 0:
            continue
        q_top_k.append([q, matches_ids, counts])

    top_k_df = pd.DataFrame(q_top_k, columns=["Query", "Top_"+str(k_val)+"_Ids", "Citation_Cnts"])

    return top_k_df


## Function that appends DataFrame rows to a CSV file
def append_data(df, file_name, is_new_file):

    if is_new_file:
        # if True, then
        df.to_csv(file_name, index=False)
    else:
        # False: This is an existing CSV file
        df.to_csv(file_name, mode='a', index=False, header=False)

    return

In [ ]:
# Load in the model for English
old_nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_ner_bionlp13cg_md")
# Can't retrieve XML if you don't have a query.
query = "GABA AND Glutamate"
Entrez.email = "n01365801@unf.edu.com"
df_q_pubmed = get_query_info_no_genes(query, "pubmed")
print("Done.")

C:\Users\danie\AppData\Roaming\Python\Python38\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
"""
Documentation for fetch_pubmed()

This function's purpose is to use NCBI's E-Utils to get the body of articles, given an id.
The E-Util used in E-Fetch.
Future Work on this could be adjusting the argument, to allow for just a list of IDs, instead of a Pandas DataFrame slice.

Arguments:
    * ids: The IDs of articles from a Pandas DataFrame
Return Value: A list of the records.
"""
def fetch_pubmed(ids): 
    i = 0
    records_pubmed = []
    # Fetch all records pertaining to our queries.
    for row in ids:
        for uid in row:
            i = i + 1
            try:
                handle = Entrez.efetch(db="pubmed", id=uid, rettype='', retmode='xml')
            except:
                print("Fetch Failure at index" + str(i))
            try:
                record = Entrez.read(handle, validate=True, escape=True)
                records_pubmed.append(record)
            except:
                print("Read Failure at index" + str(i))
                

    # Be polite and flush/close the stream like a good programmer.
    handle.close()
    return records_pubmed
print("Done.")



In [ ]:
df_q_pubmed.head()

In [ ]:
#Fetch failures due to Http error when the API is particularly busy. Errors reduce when the query is run during off hours.
records_pubmed = fetch_pubmed(df_q_pubmed['MainID_List'])
print(str(len(records_pubmed)) + ' records.')


In [ ]:
"""
Documentation for invert_dict()

This function's sole purpose is to invert a dictionary, so that the values of the old are the keys of the new,
and the keys of the old are the values of the new, in list format.

Arguments:
    dictionary: The dictionary to be inverted.
Return Value: The inverted dictionary following the above design.
"""
def invert_dict(dictionary):
    dict_inverted = {} # output
    for (k, v) in dictionary.items():
        if v in dict_inverted.keys():
            dict_inverted[v].append(k)
        else:
            dict_inverted[v] = [k]
            
    return dict_inverted
print("Done.")

In [ ]:
"""
Documentation for clean_text()

This function intakes a text string and removes punctuation that does not signal the end of a sentence.

Arguments:
    text: a string of text
Return Value: 
    clean_string: a string without punctuation
"""
def clean_text(text):
    try:
        #text_alphanumeric = re.sub(r'[^a-zA-Z0-9]', ' ', text)
        clean_string = re.sub(r'[^\w\s]', '', text)
        return clean_string
    except:
        print("error")
        return None
    
"""
Documentation for stopword_remove()

This function intakes a text string and removes english stopwords from nlp.Defaults.stop_words.

Arguments:
    text: a string of text
Return Value: 
    clean_string: a string without stopwords
"""

def stopword_remove(text):
    text_for_removal = text.split(" ")
    tokens_filtered= [word for word in text_for_removal if not word in nlp.Defaults.stop_words]
    return ' '.join(tokens_filtered)

"""
Documentation for stopword_remove()

This function is used as a preprocessing pipeline to call the clean_text() and stopword_remove() functions.

Arguments:
    text: a string of text
Return Value: 
   final_text: a string without english stopwords and non-stopping punctuation
"""

def preprocess(text):
    text_no_punct = clean_text(text)
    final_text = stopword_remove(text_no_punct)
    return final_text


In [ ]:
"""
Documentation for find_comentions()

Future Work for this function includes generalizing it to be able to handle both PubMed and PMC.
This will likely require some work on reranking() below, as it only handles PubMed formatted XML,
due to issues with PMC and Biopython present while writing this code.

Arguments:
    * thes: A dictionary containing the synonyms of query terms.
    * doc: A SpaCy Doc object that contains the text we are looking at.
Return Value: A tuple in the form of (sentences, proximity list)
"""
def find_comentions(thes, doc):
    j = 0
    sentences = []
    proximity_list = []
    count = 0
    #print(doc)
    for sentence in doc.sents:
        #print(sentence)
        prev_term = ""
        term_seen = False
        first_i = 0
        for word in sentence:
            for term in thes.keys():
                #print(term)
                if term_seen:
                    if (word.text in thes[term]) and (word.text not in thes[prev_term]):
                        count = count + 1
                        #print(word.text + "Calculated!!")
                        #print(word.i)
                        proximity_list.append(int(word.i - first_i))
                        first_i = word.i
                        prev_term = term
                        if j not in sentences:
                            sentences.append(j)
                        break
                elif (word.text in thes[term]):
                    count = count + 1
                    #print(word.text)
                    #print(word.i)
                    term_seen = True
                    first_i = word.i
                    prev_term = term
                    break
        j= j+1
    return (sentences, proximity_list, count)

def extract_comentions(doc, sentences):
    j = 0
    comention_sentences = []
    for sentence in doc.sents:
        if j in sentences and sentence.text not in comention_sentences:
            comention_sentences.append(str(sentence.text))
        j = j + 1
    return str(comention_sentences)
            
print("Done.")

In [ ]:
"""
Documentation for bm_reranking()

This function will use find_comentions(), invert_dict(), and bm25 Okapi to create a ranking of the articles.
The articles will be identified by UID.

Future additions to this reranking function include using Ms. Victoria's get_top_k() and cited_cnt_table() functions to add in the
25 citation requirement for credibility. Other work includes breaking out some functionality into other functions in order to clean up the mess.

Arguments:
    * records: A list of XML Objects returned by E-Fetch.
    * query_terms: A list of query_terms. These then get selected out of the Thesaurus.
Return Value: 
    rankings: The rankings for the records. Type is a Pandas DataFrame.
    bmtop15: Top 15 ranked records.
    top_100: Top 100 of the rankings.
"""
def bm_reranking(records, query_terms):
    # snippet from https://stackoverflow.com/questions/29216889/slicing-a-dictionary
    # While in the answer, they add a check to make sure the key is in original dict
    # (the thesaurus in this case), it is safe to assume that the key is in the dict,
    # because the thesaurus should contain all possible terms.
    inner_thesaurus = {k:thesaurus[k] for k in query_terms}
    #print(inner_thesaurus)
    # DataFrame Data
    pmids = []
    titles = []
    abstracts = []
    token_abstracts = []
    relevancy_score = {}
    abstract_score = []
    comention_sentences = []
    tokenized_queries = []
    i = 0
    j = 0
    for record in records:
        # Some records do not have an Abstract (???) so we need to check for an abstract
        # TODO: Deal with the articles that don't have abstracts
        try:
            pmid = str(record['PubmedArticle'][0]['MedlineCitation']['PMID'])
            article_keys = record['PubmedArticle'][0]['MedlineCitation']['Article'].keys()
            if 'Abstract' in article_keys: # We have an Abstract
                # TODO: Stop rewriting this indexing mess every time.
                abstract_text = str(record['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText'])
                # Process the abstract
                #doc = nlp(abstract_text)
                #Preprocess abstracts with punct and stopword removal
                abstract_text_clean = re.sub(r'/', ' ', abstract_text) #removes / characters because of unique case involving scispacy NER, combines query1/query2 into an unrecognizable token
                abstract_text_clean = stopword_remove(abstract_text_clean)
                doc = nlp(abstract_text_clean) 
                #removes punctuation that does not indicate the end of a sentence
                text_no_punct = [token.text for token in doc if not (token.is_punct or token.is_left_punct or token.is_right_punct) or (token.text ==  ("." or "!" or "?"))]
                token_abstracts.append(text_no_punct)
                
                #dataframe collection
                pmids.append(pmid)
                titles.append(str(record['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle']))
                abstract_text = "".join(str(i) for i in (record['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText']))
                abstracts.append(abstract_text)
        except:
            print("Record indexed at " + str(i) + " PMID not found.")
        i = i + 1
        j = j + 1
    
    bm25 = BM25Okapi(token_abstracts)
    
    for key in inner_thesaurus.keys():
        tokenized_queries = inner_thesaurus[key] + tokenized_queries
    
    doc_scores = bm25.get_scores(tokenized_queries)
   
    print(tokenized_queries)
    
    prerankings = pd.DataFrame(data=[pmids, titles, abstracts, doc_scores]).transpose()
    prerankings.columns = ["PMID", "Title", "Abstract", "BM25 Relevancy Score"]
    bmrankings= prerankings.sort_values(by=["BM25 Relevancy Score"], ascending=False).reset_index(drop=True)
    
    return bmrankings.iloc[:15].copy(), bmrankings

bmtop15,bmrankings = bm_reranking(records_pubmed, ["GABA","Glutamate"])

In [ ]:
bmtop15.head()

In [ ]:
def calc_relevancy_score(proximity_count):
    if (len(proximity_count) != 0):
        relevancy_score = sum([1/count for count in proximity_count])
    else:
        relevancy_score = 0
    return relevancy_score

In [ ]:
"""
Documentation for reranking()

This function will use find_comentions(), invert_dict(), and word_proximity() to create a general ranking of the articles.
The articles will be identified by UID.

Future additions to this reranking function include using Ms. Victoria's get_top_k() and cited_cnt_table() functions to add in the
25 citation requirement for credibility. Other work includes breaking out some functionality into other functions in order to clean up the mess.

Arguments:
    * records: A list of XML Objects returned by E-Fetch.
    * query_terms: A list of query_terms. These then get selected out of the Thesaurus.
Return Value: 
    rankings: The top 15 rankings for the records. Type is a Pandas DataFrame.
    prerankings: Unsorted scored dataframe of records. Used for comparison to bm25 rankings.
    top_100
    
"""
### TODO: Break apart this function into smaller functions
    # Namely, creation of combined_criteria, creation of top_15, and creation of DataFrame
def reranking(records, query_terms):
    # snippet from https://stackoverflow.com/questions/29216889/slicing-a-dictionary
    # While in the answer, they add a check to make sure the key is in original dict
    # (the thesaurus in this case), it is safe to assume that the key is in the dict,
    # because the thesaurus should contain all possible terms.
    inner_thesaurus = {k:thesaurus[k] for k in query_terms}

    # DataFrame Data
    pmids = []
    titles = []
    abstracts = []
    relevancy_score = {}
    comention_sentences = []
    raw_counts = []
    i = 0
    j = 0
    for record in records:
        # Some records do not have an Abstract (???) so we need to check for an abstract
        # TODO: Deal with the articles that don't have abstracts
        try:
            pmid = str(record['PubmedArticle'][0]['MedlineCitation']['PMID'])
            article_keys = record['PubmedArticle'][0]['MedlineCitation']['Article'].keys()
            if 'Abstract' in article_keys: # We have an Abstract
                # TODO: Stop rewriting this indexing mess every time.
                abstract_text = str(record['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText'])
                # Process the abstract
                #CHANGE using raw string for comention
                
                #FIXME find way to stop calling nlp() twice 
                #tokenize and process abstracts to extract comentions and relevancy scores
                abstract_text_clean = re.sub(r'/', ' ', abstract_text) #removes / characters because of unique case involving scispacy NER, combines query1/query2 into an unrecognizable token
                doc = nlp(abstract_text_clean) 
                #removes punctuation that does not indicate the end of a sentence
                text_no_punct = [token for token in doc if not (token.is_punct or token.is_left_punct or token.is_right_punct) or (token.text ==  ("." or "!" or "?"))]
                text_no_punct= ' '.join(token.text for token in text_no_punct)
                
                doc_no_punct= nlp(text_no_punct)
                (comention_sents_indices, proximity_count, raw_count) = find_comentions(doc=doc_no_punct, thes=inner_thesaurus)
                
                
                # DataFrame data collection
                pmids.append(pmid)
                relevancy_score[pmid] = calc_relevancy_score(proximity_count)
                raw_counts.append(raw_count)
                titles.append(str(record['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle']))
                try:
                    abstract_text = "".join(str(i) for i in (record['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText']))
                    doc = nlp(abstract_text)
                    comention_sents = extract_comentions(doc, comention_sents_indices)
                    abstracts.append(abstract_text)
                    comention_sentences.append(comention_sents)
                except:
                    print("Reformatting error at " + str(i) + ".")
                
        except:
            print("Record indexed at " + str(i) + " PMID not found.")
        i = i + 1
        j = j + 1
    print("Num records proccessed: " + str(j))
    
    # Relevancy score will be used to calculate the index.
    prerankings = pd.DataFrame(data=[pmids, titles, abstracts, relevancy_score.values(), raw_counts, comention_sentences]).transpose()
    prerankings.columns = ["PMID", "Title", "Abstract", "Relevancy Score", "Number of Query Terms Found", "Comention Sentences"]
    #FIXME: remove plotting
    
    
    
    #formatting for output
    
    relevancy_score_inverted = invert_dict(relevancy_score)
    # Gathering the top 15 abstracts based on the relevancy score.
    # We conglomerate all of the scores into a single list, flatten it, and then simply slice out the first 15 elements.
    top_15 = []
    for i in sorted(relevancy_score_inverted.keys(), reverse=True):
        top_15.append(relevancy_score_inverted[i])
    # This list comprehension is flattening the list of lists produced by combined_criteria filtering.
    top_15 = [x for xs in top_15 for x in xs]
    top_15 = top_15[:15]
    prerankings.astype({'Comention Sentences': 'string'}).dtypes
    prerankings["Comention Sentences"] = prerankings["Comention Sentences"].apply(lambda x: str(x).strip("[]"))
    
    #getting top 100 for word clouds
    top_100 = []
    for i in sorted(relevancy_score_inverted.keys(), reverse=True):
        top_100.append(relevancy_score_inverted[i])
    # This list comprehension is flattening the list of lists produced by combined_criteria filtering.
    top_100 = [x for xs in top_100 for x in xs]
    top_100 = top_100[:100]
    
    # Now that we have the UIDs of our top 15, we can grab them
    #FIXME: need to redo top 15 selections with iloc like function for BMRANKINGS
    prerankings = prerankings.sort_values(by="Relevancy Score", ascending=False).reset_index(drop=True)
    rankings = prerankings[prerankings["PMID"].isin(top_15)].sort_values(by="Relevancy Score", ascending=False).reset_index(drop=True)
    top_100ranked = prerankings[prerankings["PMID"].isin(top_100)].sort_values(by="Relevancy Score", ascending=False).reset_index(drop=True)
    return rankings, top_100ranked, prerankings

# FIXME: Do NOT hardcode the query_terms, pull them in from the query.
rankings, top_100, prerankings = reranking(records_pubmed, ["GABA","Glutamate"])


In [ ]:
prerankings.head()

In [ ]:
#needed for merge, sets index values for join
prerankings[prerankings.index.name] = prerankings.index
prerankings["Comention Score Rank"]= prerankings[prerankings.index.name]
bmrankings[bmrankings.index.name] = bmrankings.index
bmrankings["BM25 Rank"]= bmrankings[bmrankings.index.name]

In [ ]:
bmrankings_merged = pd.merge(left = bmrankings, right = prerankings, left_on = "PMID", right_on ="PMID")
bmrankings_mergedtop15 = bmrankings_merged.iloc[:15]

bmrankings_mergedtop15.head()

bm_comparison =  bmrankings_mergedtop15.filter(['PMID','Title_x','Abstract_x', 'BM25 Relevancy Score','Relevancy Score', 'Comention Score Rank'], axis=1)
bm_comparison.head()

bm_comparison.rename(columns={"Title_x": "Title", "Abstract_x": "Abstract","Relevancy Score":"Comention Relevancy Score"}, inplace = True)
bm_comparison.head()

In [ ]:
rankings_merged = pd.merge(left = prerankings, right = bmrankings, left_on = "PMID", right_on ="PMID")
rankings_mergedtop15 = rankings_merged.iloc[:15]

rankings_mergedtop15.head()

comention_comparison =  rankings_mergedtop15.filter(['PMID','Title_x','Abstract_x', 'Relevancy Score','BM25 Relevancy Score', 'BM25 Rank'], axis=1)
comention_comparison.head()

comention_comparison.rename(columns={"Title_x": "Title", "Abstract_x": "Abstract","Comention Relevancy Score":"Relevancy Score"}, inplace = True)
comention_comparison.head()

In [ ]:
#output compared rankings to other method
bm_comparison.to_csv("GABA_AND_GlutamateBM25vsComentionRanks07.csv")
comention_comparison.to_csv("ComentionvsBM25Ranks05.csv")

In [ ]:
REST_URL = "http://data.bioontology.org"
"""
Use of this requires a valid API Key.
This simply requires setting up an account at bioportal.bioonotology.org,
but do note that API Keys are meant to be secret, and probably more permanent than
what amounts to basically an intern's key.
"""
API_KEY = "0de90ddd-0da2-42ee-b0a0-2eb3b90dface"
ont = "HP,MDM"
def get_json(url):
    opener = urllib.request.build_opener()
    opener.addheaders = [('Authorization', 'apikey token=' + API_KEY)]
    return json.loads(opener.open(url).read())

abstract = "Ivermectin (IVM) is an antiparasitic drug that is widely used in domestic animals. In mammals, IVM acts as a γ-aminobutyric acid (GABA) receptor agonist. This neurotransmitter plays an important role in the regulation of female sexual behavior. The present study investigated the effects of therapeutic (0.2 mg/kg) and high (1.0 mg/kg) IVM doses on female sexual behavior in physiological and pharmacological conditions. Female rats in estrus or treated with estradiol valerate to induce sexual behavior 24 h before the experiments were used. Ivermectin was administered 15 min before the sexual observations. The number of lordosis events in 10 mounts was recorded to calculate the lordosis quotient. The intensity of lordosis (0 [no lordosis], 1 [low lordosis], 2 [normal lordosis] and 3 [exaggerated lordosis]) was scored. In estrus and hormonal treated female rats, both IVM doses decreased the intensity of the lordosis reflex and the percentage of females that presented high levels of lordosis (exaggerated lordosis). However, the number of females that presented lordosis was unaltered. We conclude that in both hormonal conditions, 0.2mg/kg IVM treatment reduced female sexual behavior and the execution of the lordosis reflex. The present results may be useful for avoiding the side effects of this drug in veterinary practice."
annotations = get_json(REST_URL + "/annotator?text=" + urllib.parse.quote(abstract) + "&ontologies=" + ont)

In [ ]:
"""
Documentation for ann_extraction()

This function's purpose is to feed the queried abstracts into different bioontology databases for annotation and extract the terms found within.
Calls get_annotations() for parsing of results and formatting.

Arguments:
    * ont: List of codes determining which ontologies to annotate from.
    * table: Pandas dataframe of records pulled from query.
Return Value: ranked_annotations, a list of dictionaries with annotated terms separated by source ontology that can be set added as columns to the dataframe of choice.
"""
def ann_extraction(ont, table):
    #i = 0
    ranked_annotations = []
    ranking = table
    for abstract in ranking["Abstract"]:
        #i = i +1
        #print("ABSTRACT " + str(i))
        annotations = (get_json(REST_URL + "/annotator?text=" + urllib.parse.quote(abstract) + "&ontologies=" 
                                + ont + "&roots_only=true&require_exact_match=true&exclude_synonyms=true"))
        ranked_annotations.append(get_annotations(annotations, ont))
         
    return ranked_annotations
    
    
"""
Documentation for get_annotations()

This function takes the returned JSON annotation objects from ann_extraction(), seperates the annotations by source ontology, and formats them for clean output.

Arguments:
    * ont: List of codes determining which ontologies to annotate from.
    * annotations: JSON formatted collection of annotation objects to be parsed for output.
Return Value: ont_dict, a dictionary with the ontology codes as keys and a list of annotated terms from the source ontology as values.
"""
def get_annotations(annotations, ont):
    ont_cat = ont.split(",")
    ont_dict={}
    for ont in ont_cat:
        ont_dict[ont] = []

    for result in annotations:
        class_details = result["annotatedClass"]
        try:
            class_details = get_json(result["annotatedClass"]["links"]["self"])
        except urllib.error.HTTPError:
            print(f"Error retrieving {result['annotatedClass']['@id']}")
            continue
        ont_dict[str(class_details["links"]["ontology"].split("/")[-1])] += [class_details["prefLabel"]]
        

    return ont_dict
"""
Documentation for attach_annotations()

This function takes the list of separated annotation dictionaries per abstract, checks for duplicates within each abstract, and formats them or attachment to the CSV.

Arguments:
    * ont: List of codes determining which ontologies to annotate from.
    * columns: List of dictionaries with separated annotations by ontology.
    * rankings: Dataframe of records from query.
Return Value: None
"""
#adds anotatons to rankings table
def attach_annotations(ont, columns, rankings):

    sources = ont.split(",")
    for source in sources:
        cols = []
        for ele in columns:
            items = []
            temp = ele[source]
            for i in temp:
                if i not in items:
                    items.append(i)
            items = str(items)
            cols.append(items.strip("[]"))
        rankings[source] = cols



In [ ]:
attach_annotations(ont, columns, rankings)
attach_annotations(ont, top100_columns, top_100)
attach_annotations(ont, bm_columns, bmtop15)
columns = ann_extraction(ont, rankings)
top100_columns = ann_extraction(ont, top_100)
bm_columns = ann_extraction(ont, bmtop15)

In [ ]:
# CSV output of individual ranakings with annotations
rankings.to_csv("Glutamate and GABA22scispacy.csv")
bmtop15.to_csv("Glutamate and GABA_BMstopwordtest8.csv")


In [ ]:
ax = prerankings["Relevancy Score"].plot(kind = 'hist', logy = True, bins = 120, title = "Distribution of Relevancy Scores",
                                    xlim = (0,12))
ax.set_xlabel("Relevancy Scores")
ax.set_ylabel("Log Frequency")


In [ ]:
def count_annotations(ont, rankings):
    i=0
    ont_ls = []
    ont_cat = ont.split(",")
    for cat in ont_cat:
        ont_dict = {}
        for record in rankings[cat]:
            annotations = record.split(",")
            for item in annotations:
                if item != '':
                    temp = item.strip(" ")
                    if temp in ont_dict.keys():
                        ont_dict[temp] = ont_dict[temp] + 1
                    else:
                        ont_dict[temp] = 1
        ont_ls.append(ont_dict)
    return ont_ls

ann_dicts = count_annotations(ont, rankings)
bm_ann_dicts = count_annotations(ont, bmtop15)

In [ ]:
def sort_dicts(ann_dicts):
    output = []
    for values in ann_dicts:
        x=dict(sorted(values.items(), key=lambda item: item[1], reverse = True))
        output.append(x)
    return output
sorted_ann_vals = sort_dicts(ann_dicts)
sorted_bm_ann_vals = sort_dicts(bm_ann_dicts)
print(sorted_ann_vals)
print(sorted_bm_ann_vals)



In [ ]:
def dict_to_table(ont, list_sorted_dict):
    sources = ont.split(",")
    i = 0
    dfs = []
    for i in range (0,len(sources)):
        vals = list_sorted_dict[i]
        anns = pd.DataFrame.from_dict(vals, orient = "index")
        anns.columns= [sources[i] + " Count"]
        dfs.append(anns)
    return dfs

def concat_dfs(ont, df_list):
    sources = ont.split(",")
    i = 0
    result = df_list[0]
    for i in range (0, len(sources)):
        try:
            result = pd.concat([result, df_list[i+1]], axis=1)
        except:
            return result
    return result
co_dfs = dict_to_table(ont, sorted_ann_vals)
bm_dfs = dict_to_table(ont, sorted_bm_ann_vals)


In [ ]:
#co_dfs[0][0:25].style
#co_dfs[1][0:25].style
#bm_dfs[0][0:25].style
#bm_dfs[1][0:25].style

In [ ]:
"""
Documentation generate_wordclouds()

Uses Wordcloud() to generate wordclouds for the annotated records.

Arguments:
    * ont: List of ontology codes.
    * rankings: Pandas dataframe of ranked items

"""
def generate_wordclouds(ont, rankings):
    sources = ont.split(",")
    text = ""
    
    for source in sources:
        temp = rankings[source]
        for item in temp:
            annotations = item.split(",")
            for element in annotations:
                text += " " + element.strip(" ''")
        wordcloud = WordCloud(max_font_size=40, max_words=100, background_color="white", normalize_plurals = True, scale = 15 ).generate(text)
        plt.figure()
        plt.imshow(wordcloud, interpolation="bilinear")
        plt.axis("off")
        plt.show()
        wordcloud = WordCloud().generate(text)
        
# Create and generate a word cloud image:
        
generate_wordclouds(ont, rankings)
generate_wordclouds(ont, bmtop15)